In [1]:

from my_libs import *


# Defining datafeed object



def backtest_pair(i,j, strategy, capital = 2000, method = "day",back_day = 80,window = 3 ):

#     try:
    robinhood = robingateway()
    
    price1 = get_price_data([i], method=method,robinhood=robinhood,back_day=back_day )
    price1 = price1.set_index("TimeStamp")
    price1.loc[price1.Open == 0,"Open"] = np.NaN
    

    price2 = get_price_data([j], method=method,robinhood=robinhood,back_day=back_day )
    price2 = price2.set_index("TimeStamp")
    price_table = pd.DataFrame()
    #--------------------------------------------------------------
#     return price1, price2
    


    price1.loc[:,"log_ret"] = log(price1.Close / price1.Close.shift(1))
    price2.loc[:,"log_ret"] = log(price1.Close / price2.Close.shift(1))


    price1.loc[:, "log_ret_mv"] = price1.log_ret.rolling(window).mean()
    price2.loc[:, "log_ret_mv"] = price2.log_ret.rolling(window).mean()

    price1 = price1.fillna(method="bfill")
    price2 = price2.fillna(method="bfill")
    
    price_table.loc[:,"relative"] = price1.log_ret_mv / price2.log_ret_mv

    price_table.loc[:,"relative_mv"] = price_table["relative"].rolling(window, min_periods=window).mean()
    price_table["relative_mv"] = abs(price_table["relative_mv"])
   

    for i in range(len(price1)-window):
        i += window
        price_table.loc[price_table.iloc[i].name,"slope"] = stats.linregress(price1.log_ret.iloc[:i],price2.log_ret.iloc[:i])[0]





    price1.loc[:,"volatility"] = price1.log_ret.rolling(window,min_periods=window-5).std()
    price2.loc[:,"volatility"] = price2.log_ret.rolling(window,min_periods=window-5).std()


    price_table["z_score"] =( price_table["relative"]-price_table["relative_mv"])/price_table.relative.std()
    price_table = price_table.fillna(method="bfill")




    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy)
    cerebro.broker.setcommission(commission=0.0)



    # Create a Data Feed
    data1 = datapairtrade(dataname=price1)
    data2 = datapairtrade(dataname=price2)
    data3 = datapairtrade_table(dataname=price_table)


    cerebro.adddata(data1)
    cerebro.adddata(data2)
    cerebro.adddata(data3)
    cerebro.broker.setcash(capital)
    #cerebro.addanalyzer(bt.analyzers.PyFolio)
    init = cerebro.broker.getvalue()
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    post = cerebro.broker.getvalue()
    if post > init:
        print ("Good Trade")

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
#     cerebro.plot()
            
#     except Exception as e:
#         print (e)
#         return

class pair_trade(bt.Strategy):
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, price: %.2f, size: %d' % (order.executed.price, order.executed.size))
            elif order.issell():
                self.log('SELL EXECUTED, %.2f, size: %d' % (order.executed.price,order.executed.size))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.data.datetime.date(0)
        print('%s, %s' % (dt, txt))
    
    
    def buy_line(self,data_frame):
        data_frame = data_frame.slope
        if data_frame>0.5:
            return -1.25
        elif data_frame>0.75:
            return -1.75
        elif data_frame:
            return -2.25
        elif data_frame<-0.75:
            return -2.75
        else:
            return -2
    def sell_line(self,data_frame):
        data_frame = data_frame.slope
        if data_frame>0.5:
            return 1.25
        elif data_frame>0.75:
            return 1.75
        elif data_frame:
            return 2.25
        elif data_frame<-0.75:
            return 2.75
        else:
            return 2


    def __init__(self):
        self.buy_lines = None
        self.sell_lines = None
        self.currentsell = 0
        self.currentbuy = 0
        #self.adxr = bt.indicators.ADXR()
        #self.cci = bt.indicators.CCI()
        #self.macd = bt.talib.MACD(self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)
        #self.mfi = bt.talib.MFI(self.data.High, self.data.Low, self.data.Close, self.data.Volume,timeperiod=14)
        #self.rocp = bt.talib.ROCP(self.data.Close, timeperiod=10)
       
        

    def next(self):
        self.buy_lines = self.buy_line(self.datas[2])
        self.sell_lines = self.sell_line(self.datas[2])
        #print (str(self.datas[2].datetime.date(0)) +" Z-score:" + str(self.datas[2].slope[0]) + " buy line: " + str(self.buy_lines) + " sell line: "+str(self.sell_lines))
   
        if self.datas[2].z_score < self.buy_lines:
            size1 = np.ceil(((self.broker.getcash()/2)/(1+self.datas[2].relative_mv[0])/self.datas[0].close[0]))
            size2 = np.ceil(((self.broker.getcash()/2) \
                                /(1+self.datas[2].relative_mv)*self.datas[2].relative_mv[0]/self.datas[1].close[0]))
            if self.getposition(self.datas[0]).size != size1:
                self.log("Close buy position: %d" % (self.getposition(self.datas[0]).size))
                self.close(data=self.datas[0])
                self.buy(size = size1,data = self.datas[0])
         
                
                
            if self.getposition(self.datas[1]).size != size2:
                self.log("Close sell position: %d" % (self.getposition(self.datas[1]).size))
                self.close(data=self.datas[1])
                
                self.buy(size = size2 ,data = self.datas[1]) # long only, change to sell when short available
                
                
                
            self.currentsell = size2

            self.log("Buy signal, Cash left:  %.2f , buy stock1: %d sell stock2:  %d"  % (self.broker.getcash(),size1,size2 ))
                
        if self.datas[2].z_score > self.sell_lines:
                
            size1 = np.ceil(((self.broker.getcash()/2)/(1+self.datas[2].relative_mv[0])/self.datas[0].close[0]))
            size2 = np.ceil(((self.broker.getcash()/2)\
                               /(1+self.datas[2].relative_mv)*self.datas[2].relative_mv[0]/self.datas[1].close[0])) 
            
            if self.getposition(self.datas[0]).size > size1:
                self.log("Close sell position: %d" % (self.getposition(self.datas[0]).size))
                self.close(data=self.datas[0])
                
                self.buy(size = size1 ,data = self.datas[0]) # long only, change to sell when short available
        
          
            if self.getposition(self.datas[1]).size != size2:
                self.log("Close buy position: %d" % (self.getposition(self.datas[1]).size))
                self.close(data=self.datas[1])
                
                self.buy(size = size2,data = self.datas[1])
           
            self.log("Buy signal, Cash left:  %.2f , sell stock1: %d buy stock2:  %d "  % (self.broker.getcash(),size1,size2 ))
        
i = "SPY"
j="VIXY"
backtest_pair(i,j,pair_trade,back_day = 80,method = "self_minute")
                    
                    